In [2]:
import pandas as pd

In [3]:
import findspark
findspark.init("/opt/manual/spark")

In [4]:
from pyspark.sql import SparkSession, functions as F

In [5]:
# If you are running Spark 2.4, you have to use Delta Lake 0.6.0.
spark = (SparkSession.builder
         .appName("Delta Lake Example")
         .master("yarn")
         .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0")
         .getOrCreate())

In [6]:
from delta.tables import *

In [7]:
# hdfs dfs -mkdir /user/train/loans_delta
deltaPath = "hdfs://localhost:9000/user/train/loans_delta"

## Create a spark dataframe

In [8]:
customers = spark.createDataFrame([(1416, "Tuncay Avcı", "9. Kısım"), 
                                   (1417, "İsmail İçtüzer", "9. Kısım"),
                                   (1506, "Mustafa Aslan", "11. Kısım"),
                                   (1527, "Buket Durmuş", "11. Kısım"),
                                   (1525, "Saadet Yılmaz", "11. Kısım")],
                                   ["Id","Name","Branch"])

In [10]:
customers.limit(5).toPandas()

,Id,Name,Branch
0,1416,Tuncay Avcı,9. Kısım
1,1417,İsmail İçtüzer,9. Kısım
2,1506,Mustafa Aslan,11. Kısım
3,1527,Buket Durmuş,11. Kısım
4,1525,Saadet Yılmaz,11. Kısım


## Write to deltalake

In [11]:
customers.write \
.mode("overwrite") \
.format("delta") \
.save(deltaPath)

## New Customers

In [13]:
customers_new = spark.createDataFrame([(1416, "Tuncay Avcı", "11. Kısım"), 
                                   (1417, "İsmail İçtüzer", "11. Kısım"),
                                   (1508, "Mustafa Bankur", "11. Kısım"),
                                   (1522, "Meliha Kaya", "11. Kısım"),
                                   (1525, "Saadet Yılmaz", "17. Kısım")],
                                   ["Id","Name","Branch"])

In [14]:
customers_new.show()

+----+--------------+---------+
|  Id|          Name|   Branch|
+----+--------------+---------+
|1416|   Tuncay Avcı|11. Kısım|
|1417|İsmail İçtüzer|11. Kısım|
|1508|Mustafa Bankur|11. Kısım|
|1522|   Meliha Kaya|11. Kısım|
|1525| Saadet Yılmaz|17. Kısım|
+----+--------------+---------+



## Read From Deltalake as DeltaTable

In [15]:
customers_delta = DeltaTable.forPath(spark, deltaPath)

In [16]:
type(customers_delta)

delta.tables.DeltaTable

In [18]:
customers_delta.toDF().show()

+----+--------------+---------+
|  Id|          Name|   Branch|
+----+--------------+---------+
|1506| Mustafa Aslan|11. Kısım|
|1527|  Buket Durmuş|11. Kısım|
|1525| Saadet Yılmaz|11. Kısım|
|1416|   Tuncay Avcı| 9. Kısım|
|1417|İsmail İçtüzer| 9. Kısım|
+----+--------------+---------+



## Upsert new customers into delta table

In [19]:
customers_delta.alias("t") \
.merge(customers_new.alias("s"), "t.Id = s.Id") \
.whenMatchedUpdateAll() \
.whenNotMatchedInsertAll() \
.execute()

In [20]:
customers_new_delta = spark.read.format("delta").load(deltaPath)

In [21]:
customers_new_delta \
.orderBy("Id") \
.show()

+----+--------------+---------+
|  Id|          Name|   Branch|
+----+--------------+---------+
|1416|   Tuncay Avcı|11. Kısım|
|1417|İsmail İçtüzer|11. Kısım|
|1506| Mustafa Aslan|11. Kısım|
|1508|Mustafa Bankur|11. Kısım|
|1522|   Meliha Kaya|11. Kısım|
|1525| Saadet Yılmaz|17. Kısım|
|1527|  Buket Durmuş|11. Kısım|
+----+--------------+---------+

